In [ ]:
input_test_p1 = """....#.....
.........#
..........
..#.......
.......#..
..........
.#..^.....
........#.
#.........
......#..."""
output_test_p1 = 41
output_test_p2 = 6

In [ ]:
import numpy as np
from collections import defaultdict

def ix(nparr):
    return [nparr[0], nparr[1]]

def parse_input(inp):
    char_dict = {"#": 1, ".": 0, "^": -1}
    g = np.array([np.array([char_dict[c] for c in line]) for line in inp.split('\n')])
    
    w = np.where(g == -1)
    start_position = np.array([w[0][0], w[1][0]])
    g[tuple(start_position)] = 0
    
    return g, start_position, "u"

def pos_in_array(arr, pos):
    for i in range(len(pos)):
        if pos[i] < 0 or pos[i] >= arr.shape[i]: return False
    return True

d_map = {"u" : np.array([-1,0]), "r" : np.array([0,1]), "d" : np.array([1,0]), "l" : np.array([0,-1])}
d_cycle = {"u":"r","r":"d","d":"l","l":"u"}

def solve_p1(inp, return_gx = False, overwrite = None):    
    g, pos, d = parse_input(inp)
    gx = np.full(g.shape, '', dtype = str)
    gx[tuple(pos)] += d
    if overwrite: g[overwrite] = 1 #used to add obstruction for p2

    while pos_in_array(g, pos):
        npos = pos + d_map[d]
        if not pos_in_array(g, npos):
            if return_gx: return gx #used as helper function for p2
            return np.sum(gx != '')
        if g[tuple(npos)] == 1:
            d = d_cycle[d]
        else:
            if d in gx[tuple(npos)]: return 'loop'
            gx[tuple(npos)] += d
            pos = npos

def next_position_fast(g, pos, d):
    if d == "u":
        l = g[:pos[0], pos[1]][::-1]
        if not np.any(l): return None
        ix = np.argmax(l != 0)
        return (pos[0] - ix, pos[1])
    if d == "d":
        l = g[pos[0]+1:, pos[1]]
        if not np.any(l): return None
        ix = np.argmax(l != 0)
        return (pos[0] + ix, pos[1])
    if d == "l":
        l = g[pos[0], :pos[1]][::-1]
        if not np.any(l): return None
        ix = np.argmax(l != 0)
        return (pos[0], pos[1] - ix)
    if d == "r":
        l = g[pos[0], pos[1]+1:]
        if not np.any(l): return None
        ix = np.argmax(l != 0)
        return (pos[0], pos[1] + ix)
    raise ValuleError        

def eval_loop(g, pos, d, pos_extra_blocker = None):
    seen = {(pos, d) : True}
    if pos_extra_blocker: g[pos_extra_blocker] = 1
    
    while True:
        pos, d = next_position_fast(g, pos, d), d_cycle[d]
        if pos == None: #exited grid
            g[pos_extra_blocker] = 0
            return False
        if (pos, d) in seen: #state revisited
            g[pos_extra_blocker] = 0
            return True 
        seen[(pos, d)] = True #remember current state
            
def solve_p2(inp):
    g, pos, d = parse_input(inp)
    gx_unobstructed = solve_p1(inp, True)
    candidate_positions = [t for t in np.ndindex(g.shape) if gx_unobstructed[t] and t != tuple(pos)]
    return sum([eval_loop(g, tuple(pos), d, cp) for cp in candidate_positions])

In [ ]:
assert solve_p1(input_test_p1) == output_test_p1
solve_p1(input_full_p1)

In [ ]:
assert solve_p2(input_test_p1) == output_test_p2
solve_p2(input_full_p1)

In [ ]:
input_full_p1 = """...............................#.........#............#...#.............................#....................#.....#........##....
..##..##.#.#....................##....#..#.....#...............#..#.....#.....................#...#.............#...#...#.........
...................#............#..................................................#............................#..#..............
............#...................................................#.........................#..#....#....................#..........
.#...........#...............................#..#....................................#............#.#............#................
.........................#..........................................#.................#...#.....#.#...............................
..#..................#......................................#......#........#.......#..........#.........#..#.........#...........
..........#..............#....................................#................................................#........#........#
..#...............................................................................................................#.#..#..........
....#....................##...................#..#....#....#.........#.......#.....................................#..............
........#...................#.......#.........................................#.........#........#....................#...........
................................................................#.......#......#...........#................#.....................
...........................................#.............#...........##.#................................#...................#....
............................#......#........#...#.................................................................................
.......#.............#...............................#.............................................#.............#.........#......
......................................................#.........................#....#................................#...........
...............#...........................................#..........#.........#..#..................#...........................
.....................................##...#................................#..#......................#.......#..........#.........
..............#.......#....................#.....................................................#.......#...................##..#
...............................................#.............#..#...#.....#..................................................#....
.....................................#........#...........................................................#......#................
........##............#...............................#....#......................................................................
.....#............#...................................................................#.............................#.............
........#...............#......#..........#........#.....#........#..........#...............................................#..#.
..................#.............#........#......#.................................................#..........................#....
.................................#............#...............................................................#..##...............
......................................#.#...........#........#.................................................................#..
...........#...........#..............#..#.#...........#............................................#...#.....................#...
....##......................#..........#..............#.............#.....#............................#....................#.....
..................#...................#..........#............................................#...#.....................#.........
.....................#...........#.........................................................#......................................
.#..............#....#..#.#...##.........#....#.............#..........#.......#...#.............................#.........#...#..
..................#...............#.....................#..#.....................#........#.......................................
......#..#......#...#..................##.............#...................#.....#.....#................................#........#.
....................#.#................#....#..................................................#............#................#....
...........................#..#.#......#........................#......#..........................................................
..#.#...............................................................#.#........................#.....................#..#.........
...#..............................#......................................................#...#....................#.#.............
....................#.......#...........#.....#.......................#........#...........................##.....................
............#.........................................................#..............................................#.........#..
..................#.#..............................#......#..................................#..........#......#..#...............
....##................#.#..................#.....#...........................................#.......................##...........
......#...........................#......#.......#..........#................#..#......#..........#..............#..#.............
........#.................#...#.........................................#..........#.........#...................................#
.#..........#....#.....#.............#...........#..................#.................#...#.......................................
...........................................................#......#...#...........................................##........#....#
............#.............................................................................#..........................#............
.........#..............................................................................................#.........#.....#......#..
.........#.......................................................................#........................#..#.....#.#............
............................#.............#..........#..#...........#......#..............#..........................#............
..#.#............#.........#.........#..................#.......................#............................................#....
...#......................#......................................................#...........#..........................#.........
.#....#...............................................................................................................#...........
.#..#................#................................................................#....^.................#.............#......
........................................................................................#.....................#...................
........##................#.............................#.........................................................................
.#.#.................#.....................#.....................#..........................#.................#...............#...
........#....................................................................................................#....................
##......#.............................#.......#........................................................................#.....##...
..........#..........................................#.................#.....#............................................#.......
......................#........................#.......................................#............#.............................
.............................#.#.......................................................................................#..........
...#.........#.....#........................#.................#...........#.....................#.................................
.........#...........................................................................#...........................#................
......#...#............#..........#........................#...................#...........#................................#.....
.....#...#....#..........................................................#......................#.....................#..........#
#.............#....................#...............................#.......#.#.............#.....................#.............#..
............................................................#.......#...................#.................................#......#
...#..#......................................#.........................................#..#.#.......##.......#............#..#....
.....#.#...#...#.#................................................................................................................
...............................................................................#........#...................#..#.........#....#...
......#.........#.............#........................................................................................#.##.......
.......................#...................................................................................................#......
.........................................................#....................................#...................##.........#....
.#..............#.#.#.............................#........................#........................#...................#.#.......
.........#.....#..........................................................................#.................#.................#...
.........#..............#...............................#.........#......#.#......................................................
......................................................................................................#....#............#.......#.
.....#.................#.......#.....#..................................#...............................................#.........
...............................................................#.............#.................................................#..
...#.....................................................................#...#.......#......#.....................................
.................................#...#.#............................................................#...#.........................
...........#..#.................................#....#...................#.............#.........#................................
..................................................#..#...........#...........................#....................................
.....#.....................................#......#......#...........................................................#............
..............#..........................................#.........#.............#..#............................#................
..........................................................................#..........#.......................#....................
.......##................#................................................................................................#.......
.#.....#.....#..................#............#......#............................#.........#......................................
..##....#..................................................................................................................#......
.......#......................#..................#...#.....................................#............#..............#..........
...............................#................#...#.................................##......................#...................
...............#..............................................................................................#.....##.......#....
................................#..........#........................................#..#..........................................
..................................................#.................................................................#.............
....................#....#.....................#.................................#.........#.....#......#......................#..
................#.............#............#................#........................................................#..........#.
.#.......#......##...................#....#..............................................................#.................#......
.........#.#............................................................................#........................#................
...............#........##.....#.....................................#..#...........#......#......................................
......#.............................................###..#.........#..............#...........................#.......#...........
.....................................#.........#...........................................................##.....................
.#....#.......................#.........#................................................................................#..#.....
....#..#.......................................................................................................##............#....
..#...................................#....#......#....................#...............................#..........................
...................#....##........................................##..........................................#...................
..........#.........#.............................................................#..#.........#...#............#.......#..#......
....#...........#...##.........................#......................#.........#..#..............................................
....................#.....#.....................................#...............................................................#.
.........#.......................#.................#...#....................................................#.#......#........#...
.................#.................................#..#..............#..........#...#.........#........#.##....##.................
.............#...........................#......................#.......................#.........................................
...............#.....#......#........#....#.......................................#.............................#.................
.............#.....##.........#...#...............................................................................................
.................................................................................#.............#.....#...#..#.....................
.................................#......#.#...#.......#......................................#......#..........................#..
#..#....................................#..#......................................................................................
...#......#.......................#..........................#...........#.#................#.#.......#...........................
#....#...............#..........................#....................#....#................................#............##........
...........#........................#..........#......................................#...................................#.....#.
......#................................#................#.............................#.........#...#........#....................
.............#................#...........#.#...........#.......................................#...........#.#...................
.....#......................#..........................#.................................#.............#............#....#........
.....#.........#.#..............................................................................#...................##............
......#.....#..#.............#...............#......#..#.#.................................#...............#...#..................
....................#.......................#...................#........................#...................#..#.....##..........
...............#...........#..#...........................................#........................#....#.........................
.....................................#...................#.#............#......................................#..................
.....#......................#.#....................#.............................................#...........................#..#.
..........#........#................#.........#...........................................#....#..#...........#..####..........#.."""